In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: aml-test
Azure region: westeurope
Subscription id: 64d7fa62-a8c4-45dc-8f71-05b6dd2a3d50
Resource group: aml-test-rg


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-07-11T19:48:32.053000+00:00', 'errors': None, 'creationTime': '2021-05-28T07:48:35.242814+00:00', 'modifiedTime': '2021-05-28T07:49:27.059192+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


In [3]:
# from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=compute_target,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_090e9bc7-db42-4bab-847f-0f452893c2e1
Web View: https://ml.azure.com/runs/HD_090e9bc7-db42-4bab-847f-0f452893c2e1?wsid=/subscriptions/64d7fa62-a8c4-45dc-8f71-05b6dd2a3d50/resourcegroups/aml-test-rg/workspaces/aml-test&tid=8f12a39d-9ae9-48cd-ac6e-8b320dc37933

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-07-25T12:18:57.663661][API][INFO]Experiment created<END>\n""<START>[2021-07-25T12:18:58.208218][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-07-25T12:18:58.573206][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_090e9bc7-db42-4bab-847f-0f452893c2e1
Web View: https://ml.azure.com/runs/HD_090e9bc7-db42-4bab-847f-0f452893c2e1?wsid=/subscriptions/64d7fa62-a8c4-45dc-8f71-05b6dd2a3d50/resourcegroups/aml-test-rg/workspaces/aml-test&tid=8f12a39d-9ae9-48cd-ac6e-8b320dc37933



In [5]:
import joblib

# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

# Get your best run and save the model from that run.

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
# get_file_names()
# Returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names())
# log_accuracy_table()
# Logs an accuracy table to the artifact store. Maybe use it for testing.
# log_accuracy_table(name, value, description='')

[{'run_id': 'HD_090e9bc7-db42-4bab-847f-0f452893c2e1_8', 'hyperparameters': None, 'best_primary_metric': 0.9176024279210926, 'status': 'Completed'}, {'run_id': 'HD_090e9bc7-db42-4bab-847f-0f452893c2e1_13', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_090e9bc7-db42-4bab-847f-0f452893c2e1_10', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_090e9bc7-db42-4bab-847f-0f452893c2e1_4', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_090e9bc7-db42-4bab-847f-0f452893c2e1_15', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_090e9bc7-db42-4bab-847f-0f452893c2e1_14', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_090e9bc7-db42-4bab-847f-0f452893c2e1_5', 'hyperparameters': None, 'best_primary_metric': 0.9

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [9]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = False)
remote_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_3f2238b2-748e-41a9-a281-e809a29ef058,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_3f2238b2-748e-41a9-a281-e809a29ef058',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-25T19:10:47.119569Z',
 'endTimeUtc': '2021-07-25T19:34:41.015618Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"64d7fa62-a8c4-45dc-8f71-05b6dd2a3d50","resource_group":"aml-test-rg","workspace_name":"aml-test","region":"westeurope","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification",

In [10]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

Package:azureml-automl-runtime, training version:1.32.0, current version:1.30.0
Package:azureml-core, training version:1.32.0, current version:1.30.0
Package:azureml-dataprep, training version:2.18.0, current version:2.15.1
Package:azureml-dataprep-native, training version:36.0.0, current version:33.0.0
Package:azureml-dataprep-rslex, training version:1.16.1, current version:1.13.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.30.0
Package:azureml-defaults, training version:1.32.0, current version:1.30.0
Package:azureml-interpret, training version:1.32.0, current version:1.30.0
Package:azureml-mlflow, training version:1.32.0, current version:1.30.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.30.0
Package:azureml-telemetry, training version:1.32.0, current version:1.30.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.30.0
Package:azureml-train-automl-runtime, training version:1.32.0, current ver

Run(Experiment: udacity-project,
Id: AutoML_3f2238b2-748e-41a9-a281-e809a29ef058_15,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=0.5208333333333334, subsample=0.6, tree_method='auto'))], verbose=False)), ('9', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=2.559547922699533, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='saga', tol=0.0001, verbose=0, warm_start=False))]

In [11]:
best_run.get_metrics()

{'AUC_micro': 0.9805283381036702,
 'recall_score_micro': 0.9158118361153262,
 'precision_score_micro': 0.9158118361153262,
 'average_precision_score_micro': 0.9812757173998821,
 'f1_score_micro': 0.9158118361153262,
 'AUC_macro': 0.946851388786857,
 'f1_score_weighted': 0.9121833396192017,
 'average_precision_score_weighted': 0.9551384640182164,
 'average_precision_score_macro': 0.8240131558100422,
 'recall_score_macro': 0.7497534088183966,
 'balanced_accuracy': 0.7497534088183966,
 'weighted_accuracy': 0.957061832149311,
 'precision_score_macro': 0.7971425008810333,
 'AUC_weighted': 0.946851388786857,
 'f1_score_macro': 0.7704577497770229,
 'matthews_correlation': 0.5447597555040484,
 'accuracy': 0.9158118361153262,
 'precision_score_weighted': 0.9100802527360765,
 'norm_macro_recall': 0.49950681763679317,
 'log_loss': 0.21205426742216316,
 'recall_score_weighted': 0.9158118361153262,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_3f2238b2-748e-41a9-a281-e809a29ef058

In [12]:
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mn...
), random_state=0, reg_alpha=0, reg_lambda=0.5208333333333334, subsample=0.6, tree_method='auto'))], verbose=False)), ('9', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=2.559547922699533, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='saga', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('4', Pipeline(memory=None, steps=[('maxabssc

In [13]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_3f2238b2-748e-41a9-a281-e809a29ef058_15,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
# Cluster clean up
compute_target.delete()